<a id="inicio"></a>
<img src="./figs/barra_uclm_esiiab.png" alt="Banner UCLM - ESIIAB" align="right">

<br><br><br>
<h1><font color="#B30033" size=5>Intelligent Systems - Course 2021-2022</font></h1>



<h1><font color="#B30033" size=5>Assignment 2: Heuristic Search Algorithms</font></h1>


<br>
<div style="text-align: left">
<font color="#4E70BE" size=3>Lecturers:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Juan Carlos Alfaro Jiménez</font><br></li>
  <li><font color="#4E70BE" size=3>Guillermo Tomás Fernández Martín</font><br></li>
  <li><font color="#4E70BE" size=3>José Antonio Gámez Martín</font><br></li>
  <li><font color="#4E70BE" size=3>Ismael García Varea</font><br></li>
  <li><font color="#4E70BE" size=3>Luis González Naharro</font><br></li>
  <li><font color="#4E70BE" size=3>Jesús Martínez Gómez</font><br></li>    
</ul>
</div>

<div style="text-align: left">
<font color="#4E70BE" size=3>Students:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Pablo Lario Gómez</font><br></li>
  <li><font color="#4E70BE" size=3>Diego Miguel López</font><br></li>   
</ul>
</div>

<br>

## Introduction
In this assignment we will study and put into practice cost-based and informed, aka heuristic, search algorithms. To do that, some algorithms studied in units 2 and 3 will be implemented and used to solve the same problem we faced with in Assignment 1: maze pathfinding.
We will also analyze and compare the performance of the algorithms by running them over different instances of the problem.

## Problem description
For the sake of completeness we also include the description of the problem in this assignment.

The maze is a grid of size N x M formed by a set of cells, some of which can be occupied by walls, which cannot be crossed. The rest of the cells will be empty and they will represent the free space. For now, the robot can only move horizontally or vertically. In addition, we can have cells with garbage, which must be cleaned by our agent, a vacuum cleaner robot.

The objective of our robot is to clean the whole area as fast as possible. In other words: **find the shortest path to find all the garbage cells in the environment**. In order to implement our robot we have to take into account that: 
- The robot can start in a random cell of the map.
- The robot can move horizontally or vertically in the maze.
- The robot can not cross walls or go beyond the limits of the maze.
- The robot will have to clean all the garbage cells found in the map, which will be automatically cleaned as soon as the robot arrives to those cells.
- For now, all the movements of the robot will have a cost of 1.
- The search will finish once all garbage cells in the map have been cleaned.

## Provided code

In the following we provide you some of the clases, implemented in `Python` that will help you to develop this assignment. 

First, we will import the necessary classes we need from the Python libraries

In [114]:
import math
import copy
import time
from queue import PriorityQueue

from abc import ABC, abstractmethod


Next, we will import some custom functions from the file `utils.py`. You don't need to modify those functions for the code to work, but feel free to have a look at them if you are curious. This code is identical to the one provided in Assignment 1.

In [115]:
from utils import *

Finally, we will import some third party libraries. We will use those to display the problem in a graphical environment. In order to do that, we will use the magic functions from jupyter to install the library from inside the notebook

In [116]:
!pip install ipythonblocks
!pip install pip
!pip install matplotlib

You should consider upgrading via the 'C:\Users\49427234\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\49427234\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\49427234\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [117]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output

In order to complete the requested search algorithms, we provide you some fundamental classes: 

#### Class `Action`
This class provides the **representation of the actions** that will be performed by the robot. You don't have to modify the code of this class. The possible actions will be: "UP", "DOWN", "RIGHT", "LEFT". This class is identical to the one provided in Assignment and, for the moment, the cost of all actions is kept at the value of 1.0.

In [118]:
class Action:
    #actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, move):
        self.move = move

    def __str__(self):
        return f"({self.move})"

    def getCost(self):
        return 1.0

#### Class `State`. 
This class provides the **representation of a state** in the search space. In this problem, a state is defined by the position of the robot and the garbages left in the maze. Note that the maze itself does not need to be part of the state given that it does not change during the search, i.e. walls are fixed during the search. You don't have to modify the code of this class.

The class `State` has an `applyAction` method that, given a valid `Action`, returns a new `State` with the action applied. This class is identical to the one provided in Assignment 1.

In [119]:
class State:

    def __init__(self, pos, garbage):
        self.pos = pos
        self.garbage = garbage

    # equals method. Returns true if the states are the same. 
    # Used for the hash table comparison, compares if both states are equal
    def __eq__(self, state):
        return self.pos == state.pos and self.garbage == state.garbage

    def __str__(self):
        return f"Position: {self.pos}\nGarbage: {self.garbage}"

    # hash method. Useful to index data structures that uses a hash function 
    #    to index elements, i.e. a Set()
    def __hash__(self):
        h = 0
        for g in self.garbage:
            h += (math.pow(10,3) * (self.pos[0]+1) + (self.pos[1]+2))
        return int(h)

    def applyAction(self, action):
        st = copy.deepcopy(self)
        
        if (action.move == "UP"):
            st.pos = (st.pos[0]-1,st.pos[1])
        elif (action.move == "DOWN"):
            st.pos = (st.pos[0]+1,st.pos[1])
        elif (action.move == "RIGHT"):
            st.pos = (st.pos[0],st.pos[1]+1)
        elif (action.move == "LEFT"):
            st.pos = (st.pos[0],st.pos[1]-1)
        else:
            print("\n*** ERROR ***: Action " + action + "  is not allowed .....\n")
            sys.exit()
            
        # if the new position has garbage clean it
        if (st.pos in st.garbage):
            st.garbage.remove(st.pos)
        
        return st

#### Class `Node`. 
This class provides a **representation of a node** in the search tree/graph. It contains the state it represents, its parent node and the action taken to reach the current node. You don't have to modify this class. 

The class `Node` also has some methods to provide access to its attributes.

**This implementation slightly differs** from the one provided in Assignment 1. Concretely:
- In addition to the `self.gGost` (cost to reach the node), two new attributes are defined: `self.hCost` and `self.fCost`, which are needed to store the heuristic and total cost of a specific node. These attributes should be conveniently assigned during the search process, accoding to the specific search strategy.
- Correspondingly two new methods are included: `getHCost(self)` and `getFCost(self)`
- The `__hash__` method, is a special method useful to index data structures that uses a hash function to index elements.
- The `__lt__` method, is a special method that defines the behaviour of the less-than operator (`<`) for objects of this class. In our case it is defined according to the value of the attribute `self.fCost`.

In [120]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state # Must be State Class
        self.parent = parent # Must be Node Class
        self.action = action # Must be Action Class
        self.depth = 0
        self.gCost, self.hCost, self.fCost = 0.0, 0.0, 0.0

    def __str__(self):
        return f"depth: {self.depth} fcost: {self.fCost}, ({self.gCost} + {self.hCost}), state: {self.state}"

    # equal method. Returns true if the states are the same. Used for the hash table comparison
    # compares if both states are equals
    def __eq__(self, other):
        if not isinstance(other, Node):
        # don't attempt to compare against unrelated types
            return NotImplemented
        return self.state == other.state

    # hash method. Useful to index data structures that uses a hash function 
    # to index elements, i.e. a Set()
    def __hash__(self):
        h = 0
        for g in self.state.garbage:
            h += (math.pow(10,3) * (self.state.pos[0]+1) + (self.state.pos[1]+2))
        return int(h)

    def __lt__(self, other):
        if not isinstance(other, Node):
            # don't attempt to compare against unrelated types
            return NotImplemented
        return self.fCost < other.fCost

    def getState(self):
        return self.state

    def getAction(self):
        return self.action
    
    def getParent(self):
        return self.parent

    def getDepth(self):
        return self.depth
    
    def getGCost(self):
        return self.gCost
    
    def getHCost(self):
        return self.hCost

    def getFCost(self):
        return self.fCost
    

## Implementation
In the following we provide you some classes and pieces of code that you will have to complete as a part of this assignment.


#### Class `Problem`
As in Assignment 1, this class provides the **representation of the search problem**. It contains the size of the maze (`rows` and `cols`), the `initialState` and the `maze`. This class can read from a file an instance of the problem to solve, or it can generate a random instance using the size of the grid, the `seed`, the maximum number of divisions (walls) in the maze and the list of garbage cells. You don't have to modify this class. 

The class `Problem` also has some methods to provide access to the initial state, the possible actions and to check if a specific state is the final/goal.

##### Heuristic functions
In this assignment you have to implement a new method, `computeHeuristic(self, st)`, in this class. This method should compute the heuristic cost of a given state, passed as argument. We have included this method in the `Problem` class because typically the computation of the heuristic cost (or distance) highly depends on the problem to solve.

It is mandatory to define and implement at least two different heuristic functions and to empirically compare them. This study should be included in the final report of this assignments, but you have to be sure that both heuristic functions are admissible and consistent. The proof of admissibility and consistency of all proposed heuristic functions also has to be included in the final report.

In [121]:
class Problem:

    actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, rows, cols, seed, maxDivisions, garbageCount, filename=""):

        if (filename != ""):
            self.rows, self.cols, self.maze = readProblemInstance(filename)
            print('Problem read with size',rows,'x',cols)
        else:
            self.rows = rows
            self.cols = cols
            self.maze = getProblemInstance(rows, cols, maxDivisions, garbageCount, seed)

        self.garbage = []

        for r in range(rows):
            for c in range(cols):
                if self.maze[r][c] == 2:
                    init_point = (r,c)
                elif self.maze[r][c] == 3:
                    self.garbage.append((r,c))

        self.initialState = State(init_point, self.garbage)

    def getInitialState(self):
        return self.initialState

    def getActions(self):
        return self.actions

    def isGoal(self, st):
        '''
        check if the given state is final or not
        '''
        return st.garbage == []
    
    def computeHeuristic(self, st, heuristic):
        '''
        Compute and return the heuristic cost from the state 'st' to the 'goal state'.
        @param st the state
        @param heuristic is the name of the heuristic to use, useful if more than one is implemented 
        '''
        maxDst = 0
        aux = list((0, 0))

        if (heuristic == "Manhattan"):
          for gb in st.garbage:
              if((abs(gb[0] - st.pos[0] + abs(gb[1] - st.pos[1])) > maxDst)):
                  maxDst = abs(gb[0] - st.pos[0]) + abs(gb[1] - st.pos[1])
        elif (heuristic == "Galletas"):
          return len(st.garbage)
        elif (heuristic == "Javivi"):
          for gb in st.garbage:
              if((abs(gb[0] - st.pos[0] + abs(gb[1] - st.pos[1])) > maxDst)):
                  maxDst = abs(gb[0] - st.pos[0]) + abs(gb[1] - st.pos[1])
                  aux = gb

          if (aux[0] == st.pos[0]): # Garbage is in the same row as the node

            if (aux[1] > st.pos[1]): # Garbage is to the right of the node
              for y in range(st.pos[1], aux[1] + 1):
                if (self.maze[st.pos[0]][y] == 1): # We penalize because there is a wall
                  maxDst += 2

            else: # Garbage is to the left of the node
              for y in range(aux[1], st.pos[1] + 1): 
                if (self.maze[st.pos[0]][y] == 1): # We penalize because there is a wall
                  maxDst += 2

          elif (aux[1] == st.pos[1]):

            if (aux[0] > st.pos[0]): # Garbage is below
              for x in range(st.pos[0], aux[0] + 1):
                if (self.maze[x][st.pos[1]] == 1):
                  maxDst += 2

            else: # Garbage is above
              for x in range(aux[0], st.pos[0] + 1):
                if (self.maze[x][st.pos[1]] == 1):
                  maxDst += 2

        return maxDst


#### Class `Search`

The `Search` class is an abstract class that contains some attributes:
- The `problem`to solve
- The list of `open` nodes, i.e. nodes in the frontier. This list is undefined given that you have to use a priority queue data structure to implement that list. Remember that nodes will be inserted in order in this list, where the order is defined according to different criteria depending on each specific search algorithm. 
- The list of `closed` nodes to implement the graph search, which is implemented using a `set` data structure.
- The attributes to account for the number of generated and expanded nodes, as well as the maximum size of the open 'list' to know the maximum number of nodes stored simultaneously in memory. Theses attributes are helpful to estimate the time and memory complexity of the algorithms.

This class also provides three methods:
- `insertNode(self, node)`: this is an abstract method that has to be implemented by all (search) classes that inherit from `Search`. You must program this method according to every specific search strategy used.
- `getSuccesors(self, node)`: this method implements the successors function and should return a list with all the valid successors of a given node. You must program this method.
- `doSearch(self)`: this method implements the graph search you have studied in class. You must program this method. It also provides some statistics of the search process.

Please, note that on the contrary to uninformed search, in heuristic search you can use the "content" of the garbage list (i.e. their positions). Also, think about the use of a data structure to conveniently manage the `open` list of nodes.

In [122]:
class Search(ABC):

    def __init__(self,problem, heuristic):
        self.problem = problem
        self.open = PriorityQueue() 
        self.closed = set()
        self.heuristic = heuristic

        self.exploredNodes = 0
        self.generatedNodes = 0
        self.expandedNodes = 0
        self.openMaxSize = 0
        self.cost = 0

    @abstractmethod
    def insertNode(self, node):
        pass

    def getSuccesors(self, node):

        suc = []
        self.expandedNodes += 1
        
        for a in self.problem.getActions():
            action = Action(a)
            auxState = node.getState().applyAction(action)

            # We check if the state is inside the maze and not out of bounds
            if (auxState.pos[0] < self.problem.rows and 
                auxState.pos[0] >= 0 and
                auxState.pos[1] < self.problem.cols and
                auxState.pos[1] >= 0):
              cell = self.problem.maze[auxState.pos[0]][auxState.pos[1]]
              
              # We check if the new state will not lead to a wall
              if cell != 1:
                nodeAux = Node(node.state.applyAction(action), node, action)
                nodeAux.gCost = node.getGCost() + 1
                nodeAux.hCost = self.problem.computeHeuristic(nodeAux.getState(), heuristic)
                nodeAux.fCost = nodeAux.gCost + nodeAux.hCost
                # if (node != None):
                nodeAux.depth = node.getDepth() + 1
                suc.append(nodeAux)
                self.generatedNodes += 1
        
        return suc

    def doSearch(self):
        totalCost = 0

        # Initial node
        currentNode = Node(self.problem.getInitialState(), None, None)
        currentNode.gCost = 0.0
        currentNode.hCost = self.problem.computeHeuristic(self.problem.getInitialState(), heuristic)
        currentNode.fCost = currentNode.gCost + currentNode.hCost
        self.insertNode(currentNode)
        
        self.generatedNodes += 1
        
        actionSequence = []

        finish = False

        while (True):
          
          # No more nodes that we can open, then we exit the loop
          if (self.open.empty()):
            break

          # We get the node with the least cost (the cost (index) used depends on the algorithm)
          currentNode = self.open.get()[1]

          self.exploredNodes += 1
          if (self.problem.isGoal(currentNode.getState())):
            self.closed.add(currentNode.getState())
            finish = True
            break

          # We check whether the node we have picked from open has been explored
          if currentNode.getState() not in self.closed: 
            suc = self.getSuccesors(currentNode)
            for s in suc:
              self.insertNode(s)

          self.closed.add(currentNode.getState())
        
        if (finish):
          print("Depth of the solution: " + str(currentNode.getDepth()))

          while (currentNode.getParent() != None):
            actionSequence.append(currentNode.getAction())
            currentNode = currentNode.getParent()
            self.cost += 1

        print("Generated nodes: " + str(self.generatedNodes))  
        print("Expanded nodes: " + str(self.expandedNodes))  
        print("Explored nodes: " + str(self.exploredNodes))
        print("Solution cost: " + str(self.cost))
        
        #i = ""
        for i in actionSequence:
          print(i)

        return actionSequence
    

#### Class `UniformCost`, `BestFirst` and `AStar`

These three classes also inherit from `Search` and will implement the uniform cost, best first and $A^*$ search strategies, respectively. You have to implement these three classes.

Despite the `UniformCost` algorithm belongs to the class of non-informed search algorithms, we have included in this assignment because of the similarity in the implementation with the informed search algorithms.

Actually the main difference betweeen these three algorithms is the way in which the function cost for a specific node ($f(n) = g(n) + h(n)$) is computed. Assuming that $g(n)$ is the real accumulated cost from the **initial state** to `n.getState()` and that $h(n)$ is the heuristic cost from `n.getState()` state to the **goal state**, $f(n)$ is computed as:

- Uniform cost: $f(n) = g(n)$
- Best First: $f(n) = h(n)$
- A$^*$: $f(n) = g(n) + h(n)$

As in Assignment 1, once the `getSuccessors(self,node)` and `doSearch(self)` methods have been implemented in the parent class, we only have to implement the `insertNode(self, node)` method, which will insert the `node`into the `self.open` list of nodes according to the corresponding values of the cost function.

In [123]:
class UniformCost(Search):
    def insertNode(self, node):
        self.open.put((node.getGCost(), node))

In [124]:
class BestFirst(Search):
    def insertNode(self, node):
        self.open.put((node.getHCost(), node))

In [125]:
class AStar(Search):
    def insertNode(self, node):
        self.open.put((node.getFCost(), node))

#### Classes `DepthFirst`, `BreadthFirst` and `DepthLimited`

These classes, implemented in Assignment 1, also can inherit from `Search` and implement the different non-informed search techniques already studied. These classes can also be used in this assigment in order to compare results and behaviours with informed search algorithms. In that case you will also have to define 
the corresponding `insertNode(self, node)` functions using the priority queue data structure and think about what value should be used for every algorithm to insert elements in the correct order to maintain the correct behaviour of every algorithm.

In [126]:
class DepthFirst(Search):
    def insertNode(self, node):
        self.open.put((0,node))

class BreadthFirst(Search):
    def insertNode(self, node):
        self.open.put((node.getDepth(), node))

class DepthLimited(Search):
    def insertNode(self, node):
        depth_limit = 5
        if node.getDepth() < self.depth_limit:
            self.open.put((node.getDepth() * -1, node))

#### The `main` function

Next, we provide you the `main` function that creates the problem and solves it using the search algorithm provided. This method should be used afterwards to carry out the experimentation to study the behaviour of the implemented algorithms for different values of the parameters provided (size of the maze, maximum number of walls, number of garbage cells, and algorithm).

In [127]:
def main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm, configFile=""):
    problem = Problem(rows, cols, seed, maxDivisions, garbageCount, configFile)

    # print("Rows: " + str(problem.rows))
    # print("Cols: " + str(problem.cols))
    # print("Seed: " + str(seed))
    # print("MaxDivisions: " + str(maxDivisions))
    # print("Garbage Count: " + str(garbageCount))
    # print("Algoritm: " + algorithm)

    if (printProblemInstance):
      print("Problem instance:")
      printMaze(problem.maze)
      print("")

    search = None

    if algorithm == "BreadthFirst":
        search = BreadthFirst(problem, heuristic)
    elif algorithm == "DepthFirst":
        search = DepthFirst(problem, heuristic)
    elif algorithm == "DepthLimited":
        search = DepthLimited(problem, heuristic)
    elif algorithm == "UniformCost":
        search = UniformCost(problem, heuristic)
    elif algorithm == "BestFirst":
        search = BestFirst(problem, heuristic)
    elif algorithm == "AStar":
        search = AStar(problem, heuristic)
    else:
        raise Exception

    time_start = time.perf_counter()
    path = search.doSearch()
    time_end = time.perf_counter()
    print("")
    print("Elapsed time: " + str(time_end - time_start) + " seconds")

    if (printProblemInstance):
      return path, problem

    return path

#### Test your code

Here you have a piece of code to test your implementation. For example, the code to execute a random search technique can be:

#### Printing the result

Here we provide you some code to display the maze and the path carried out by the robot (the green cell) to solve the instance of the problem. Walls are represented as black cells, and garbage with brown cells.

In [128]:
def render_maze(grid, maze, garbage):
    height, width = len(maze), len(maze[0])
    # Render maze
    for i in range(width):
        for j in range(height):
            grid[j,i] = (200,200,200) if maze[j][i] in [0,2,3] else (0,0,0)

    # Render garbage in maze
    for g in garbage:
        grid[g[0],g[1]] = (139,69,19)
        
def find_agent(maze, width, height):
    for i in range(width):
        for j in range(height):
            if (maze[j][i] == 2):
                return (i,j)

def render_path(path, maze, garbage):
    height, width = len(maze), len(maze[0])
    solution_grid = BlockGrid(width, height, fill=(200, 200, 200))
    
    movementDict = {'DOWN':(0,1), 'UP':(0,-1), 'LEFT':(-1,0), 'RIGHT':(1,0)}
    agentPos = find_agent(maze, width, height)
    garbageRender = copy.deepcopy(garbage)
    
    # Initial position rendering
    render_maze(solution_grid, maze, garbageRender)
    solution_grid[agentPos[1],agentPos[0]] = (0, 255, 0)
    solution_grid.show()
    clear_output(wait=True)
    time.sleep(0.1)
    
    for action in path:
        # Update agent position
        agentPos = (agentPos[0] + movementDict[action.move][0], agentPos[1] + movementDict[action.move][1])
        
        # Update garbage list
        if ((agentPos[1], agentPos[0]) in garbageRender):
            garbageRender.remove((agentPos[1], agentPos[0]))
        
        # Render maze
        render_maze(solution_grid, maze, garbageRender)
        
        # Render agent, and update its position and the garbage list
        solution_grid[agentPos[1],agentPos[0]] = (0,255,0)
        
        solution_grid.show()
        clear_output(wait=True)
        time.sleep(1)

In [129]:
# render_path(path_sol, problem_instance.maze, problem_instance.garbage)

You can easily try different instances of the problem just by changing the parameters when you call both the `main(...)` function and the `render_path(...)` function:

In [130]:
# path_sol, problem_instance = main(15, 15, 2021, 15, 6, 'BestFirst')

In [131]:
# render_path(path_sol, problem_instance.maze, problem_instance.garbage)

## Experimental results

Once the algorithms have been implemented, you must study their performance. In order to do that, you must compare the quality of the solutions obtained, as well as the number of expanded nodes for instances of different maze sizes, number of walls and number of garbage cells.

Please, use new cells to insert code to carry out the experimental results and study of the algorithms.

In [132]:
rows = 25
cols = 25
seed = 102
maxDivisions = 20
garbageCount = 6
algorithm = "BestFirst"
heuristic = "Javivi"
printProblemInstance = False

# path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

In [133]:
rows = 25
cols = 25
seed = 102
maxDivisions = 20
garbageCount = 6
algorithm = "BestFirst"
heuristic = "Galletas"
printProblemInstance = False

# path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

In [134]:
rows = 25
cols = 25
seed = 102
maxDivisions = 20
garbageCount = 6
algorithm = "BestFirst"
heuristic = "Manhattan"
printProblemInstance = False

# path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

In [135]:
rows = 25
cols = 25
seed = 102
maxDivisions = 20
garbageCount = 6
algorithm = "AStar"
heuristic = "Manhattan"
printProblemInstance = False

# path_sol = main(printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

In [136]:
rows = 25
cols = 25
seed = 102
maxDivisions = 20
garbageCount = 6
algorithm = "AStar"
heuristic = "Javivi"
printProblemInstance = False

# path_sol = main(printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

In [137]:
# Testing

rows = 5
cols = 5
seed = 1
maxDivisions = 15
garbageCount = 1
algorithm = "UniformCost"
heuristic = "Manhattan"
printProblemInstance = False

'''
x = 0
y = 0
for x in range(rows, 10):
  for y in range(cols, 10):
    for z in range(seed, 5):
      for i in range(garbageCount, 2):
        path_sol = main(heuristic, printProblemInstance, x, y, z, maxDivisions, i, algorithm)
        '''

'\nx = 0\ny = 0\nfor x in range(rows, 10):\n  for y in range(cols, 10):\n    for z in range(seed, 5):\n      for i in range(garbageCount, 2):\n        path_sol = main(heuristic, printProblemInstance, x, y, z, maxDivisions, i, algorithm)\n        '

<h2> ASSIGNMENT 2 - EXAM QUESTIONNAIRE - PABLO LARIO GÓMEZ</h2>

<h2>QUESTION 3</h2>

In [138]:
rows = 12 
cols = 14 
seed = 777 
maxDivisions = 8 
garbageCount = 6 
algorithm = "UniformCost"
heuristic = "None"
printProblemInstance = False

path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

Depth of the solution: 38
Generated nodes: 18296
Expanded nodes: 5800
Explored nodes: 17011
Solution cost: 38
(RIGHT)
(RIGHT)
(RIGHT)
(UP)
(RIGHT)
(RIGHT)
(RIGHT)
(LEFT)
(LEFT)
(DOWN)
(DOWN)
(DOWN)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(DOWN)
(DOWN)
(DOWN)
(DOWN)
(DOWN)
(UP)
(UP)
(LEFT)
(LEFT)
(UP)
(UP)
(UP)
(LEFT)
(LEFT)
(LEFT)
(RIGHT)
(UP)
(UP)
(UP)
(LEFT)
(UP)
(UP)

Elapsed time: 0.9052391999866813 seconds


<h2>QUESTION 4</h2>

In [139]:
rows = 12 
cols = 14 
seed = 777 
maxDivisions = 8 
garbageCount = 6
algorithm = "BestFirst"
heuristic = "Manhattan"
printProblemInstance = False

path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

Depth of the solution: 47
Generated nodes: 3785
Expanded nodes: 1175
Explored nodes: 3420
Solution cost: 47
(UP)
(UP)
(LEFT)
(LEFT)
(LEFT)
(UP)
(LEFT)
(UP)
(UP)
(LEFT)
(RIGHT)
(DOWN)
(RIGHT)
(RIGHT)
(DOWN)
(DOWN)
(RIGHT)
(RIGHT)
(UP)
(UP)
(UP)
(RIGHT)
(RIGHT)
(RIGHT)
(RIGHT)
(DOWN)
(RIGHT)
(RIGHT)
(UP)
(UP)
(UP)
(UP)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(DOWN)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(LEFT)
(UP)
(UP)
(UP)

Elapsed time: 0.20547230006195605 seconds


<h2>QUESTION 5</h2>

In [140]:
rows = 12 
cols = 14 
seed = 777 
maxDivisions = 8 
garbageCount = 6 
algorithm = "AStar"
heuristic = "Javivi"
printProblemInstance = False

path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

Depth of the solution: 38
Generated nodes: 5224
Expanded nodes: 1604
Explored nodes: 4476
Solution cost: 38
(RIGHT)
(UP)
(RIGHT)
(RIGHT)
(RIGHT)
(RIGHT)
(RIGHT)
(LEFT)
(LEFT)
(DOWN)
(DOWN)
(DOWN)
(LEFT)
(LEFT)
(LEFT)
(DOWN)
(LEFT)
(DOWN)
(DOWN)
(DOWN)
(DOWN)
(UP)
(UP)
(LEFT)
(LEFT)
(UP)
(UP)
(UP)
(LEFT)
(LEFT)
(LEFT)
(RIGHT)
(UP)
(UP)
(LEFT)
(UP)
(UP)
(UP)

Elapsed time: 0.2996505999471992 seconds


<h2>QUESTION 10 - 1</h2>

In [142]:
rows = 25 
cols = 25 
seed = 777
maxDivisions = 8 
garbageCount = 25 
algorithm = "AStar"
heuristic = "Javivi"
printProblemInstance = False

path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

KeyboardInterrupt: 

<h2>QUESTION 10 - 2</h2>

In [143]:
rows = 25 
cols = 25 
seed = 777
maxDivisions = 8 
garbageCount = 25 
algorithm = "BestFirst"
heuristic = "Javivi"
printProblemInstance = False

path_sol = main(heuristic, printProblemInstance, rows, cols, seed, maxDivisions, garbageCount, algorithm)

KeyboardInterrupt: 